In [77]:
import pickle
import json
import mmcv
from tqdm import tqdm_notebook as tqdm
from mmdet.datasets import build_dataloader, build_dataset
from pycocotools.coco import COCO
import pycocotools.coco as cocoapi
import pycocotools.mask as MASK
import os
import poly.polyiou as polyiou
import cv2
from icecream import ic
import matplotlib.pyplot as plt
from mmdet.models import build_detector
from mmcv.runner import load_checkpoint, get_dist_info
import numpy as np
from multiprocessing import Pool
from mmdet.ops.nms import nms_wrapper

In [95]:
def get_ann(bboxes, segs, cls, name, locx, locy, scale_factor):
    Rect = []
    Bbox = []
    Vis = []
    bbox_cls = bboxes[cls]
    seg_cls = segs[cls]
    if( len(bbox_cls) > 0):
        for bbox, rle in zip(bbox_cls, seg_cls):
            xmin, ymin, xmax, ymax, score = bbox
            xmin += locx
            ymin += locy
            xmax += locx
            ymax += locy
            bounding_box = np.array([xmin*scale_factor, ymin*scale_factor, xmax*scale_factor, ymax*scale_factor, score])
            mask = MASK.decode(rle)     
            contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
                                                             cv2.CHAIN_APPROX_SIMPLE)
            areas = [cv2.contourArea(contour)  for contour in contours]
            if(len(areas) > 0):
                index = np.argmax(areas)
                contour = contours[index]
                rect = cv2.minAreaRect(contour)
                box = cv2.boxPoints(rect)
                box += np.array([locx, locy])


                center = np.array(rect[0])
                size = np.array(rect[1])
                angel = rect[2]
                size *= scale_factor
                center += np.array([locx, locy])
                center *= scale_factor
                rect = (tuple(center), tuple(size), angel )
            else:
                continue
            Rect.append(rect)
            Bbox.append(bounding_box)
            Vis.append(np.array([box[0]*scale_factor, box[1]*scale_factor, box[2]*scale_factor, box[3]*scale_factor]))
    return  cls, name, Rect, Bbox, Vis

    
def get_all_ann(filename, result,img_prefix, size, CLASS_NUM=18):
    items = filename.split("_")
    name = items[0]
    img = cv2.imread(img_prefix + filename)
    h, w = size
    scale_factor = 1/float(items[1])
    locx = int(float(items[2]))
    locy = int(float(items[3]))
    bboxes = result[0]
    segs = result[1]
    Rect = [[] for cls in range(CLASS_NUM)]
    Bbox = [[] for cls in range(CLASS_NUM)]
    Vis = [[] for cls in range(CLASS_NUM)]
    for cls in range(CLASS_NUM):
#         if(cls ==7):
#             if not(locx == 0 and locy==0):
#                 continue
        bbox_cls = bboxes[cls]
        seg_cls = segs[cls]
        if( len(bbox_cls) > 0):
            for bbox, rle in zip(bbox_cls, seg_cls):
                xmin, ymin, xmax, ymax, score = bbox
                xmin += locx
                ymin += locy
                xmax += locx
                ymax += locy
                bounding_box = np.array([xmin*scale_factor, ymin*scale_factor, xmax*scale_factor, ymax*scale_factor, score])
                mask = MASK.decode(rle)     
                contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
                                                                 cv2.CHAIN_APPROX_SIMPLE)
                areas = [cv2.contourArea(contour)  for contour in contours]
                if(len(areas) > 0):
                    index = np.argmax(areas)
                    contour = contours[index]
                    rect = cv2.minAreaRect(contour)
                    box = cv2.boxPoints(rect)
                    box += np.array([locx, locy])


                    center = np.array(rect[0])
                    size = np.array(rect[1])
                    angel = rect[2]
                    size *= scale_factor
                    center += np.array([locx, locy])
                    center *= scale_factor
                    rect = (tuple(center), tuple(size), angel )
                else:
                    continue
                Rect[cls].append(rect)
                Bbox[cls].append(bounding_box)
                Vis[cls].append(np.array([box[0]*scale_factor, box[1]*scale_factor, box[2]*scale_factor, box[3]*scale_factor]))
    return  name, Rect, Bbox, Vis


In [83]:
def merge_result(config_file, result_file, anno_file, img_prefix, out_file=None, CLASS_NUM=18):
    cfg = mmcv.Config.fromfile(config_file)
    results = mmcv.load(result_file)
    dataset = build_dataset(cfg.data.test)
    img_infos = dataset.load_annotations(anno_file)
    ann = {}
    rets= []
    pbar = tqdm(total=len(results))
    def update(*a):
        pbar.update()
    p = Pool(18)
    for i in range(len(results)):
        filename = img_infos[i]['filename']
        h = img_infos[i]['height']
        w = img_infos[i]['width']
        rets.append(p.apply_async(get_all_ann, args=(filename, results[i], img_prefix, (h, w), 18), callback=update)) 

    for ret in rets:
        name, Rect, Bbox, Vis = ret.get()
        if name not in ann:
            ann[name] = {"bbox": [[] for i in range(CLASS_NUM)], "vis": [[] for i in range(CLASS_NUM)], 
                         "rect": [[] for i in range(CLASS_NUM)]}
        for cls in range(CLASS_NUM):
            ann[name]['bbox'][cls] += Bbox[cls]
            ann[name]['rect'][cls] += Rect[cls]
            ann[name]['vis'][cls] += Vis[cls]
    if out_file is not None:
        mmcv.dump(ann, out_file)
    p.close()
    p.join()
    return ann
def poly_nms(dets, thresh):
    obbs = dets[:, 0:-1]
    x1 = np.min(obbs[:, 0::2], axis=1)
    y1 = np.min(obbs[:, 1::2], axis=1)
    x2 = np.max(obbs[:, 0::2], axis=1)
    y2 = np.max(obbs[:, 1::2], axis=1)
    scores = dets[:, 8]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    polys = []
    for i in range(len(dets)):
        tm_polygon = polyiou.VectorDouble([dets[i][0], dets[i][1],
                                            dets[i][2], dets[i][3],
                                            dets[i][4], dets[i][5],
                                            dets[i][6], dets[i][7]])
        polys.append(tm_polygon)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        ovr = []
        i = order[0]
        keep.append(i)
        # if order.size == 0:
        #     break
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        # w = np.maximum(0.0, xx2 - xx1 + 1)
        # h = np.maximum(0.0, yy2 - yy1 + 1)
        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        hbb_inter = w * h
        hbb_ovr = hbb_inter / (areas[i] + areas[order[1:]] - hbb_inter)
        # h_keep_inds = np.where(hbb_ovr == 0)[0]
        h_inds = np.where(hbb_ovr > 0)[0]
        tmp_order = order[h_inds + 1]
        for j in range(tmp_order.size):
            iou = polyiou.iou_poly(polys[i], polys[tmp_order[j]])
            hbb_ovr[h_inds[j]] = iou
            # ovr.append(iou)
            # ovr_index.append(tmp_order[j])

        # ovr = np.array(ovr)
        # ovr_index = np.array(ovr_index)
        # print('ovr: ', ovr)
        # print('thresh: ', thresh)
        try:
            if math.isnan(ovr[0]):
                pdb.set_trace()
        except:
            pass
        inds = np.where(hbb_ovr <= thresh)[0]

        # order_obb = ovr_index[inds]
        # print('inds: ', inds)
        # order_hbb = order[h_keep_inds + 1]
        order = order[inds + 1]
        # pdb.set_trace()
        # order = np.concatenate((order_obb, order_hbb), axis=0).astype(np.int)
    return keep

def nms(ann, mode, thresh, out_file=None, CLASS_NUM=18):
    for name in tqdm(ann.keys()):
        info = ann[name]
        for cls in range(CLASS_NUM):
            bbox = np.array(info['bbox'][cls], np.float32)
            vis = np.array(info['vis'][cls])
            if(len(bbox)<=0):
                continue
            if mode == "rec":
                _, inds = nms_wrapper.nms(bbox, thresh)
            elif mode == "poly":
                dets = vis.reshape(-1, 8)
                dets = np.array(dets, np.int32)
                scores = bbox[:, 4]
                dets = np.c_[dets, scores]
                #print(bbox.shape)
                inds = poly_nms(dets, thresh)
           # print(len(inds))
            ann[name]['bbox'][cls] = bbox[inds]
            ann[name]['vis'][cls] = vis[inds]
    if out_file is not None:
        mmcv.dump(ann, out_file)
    return ann

def generate_submit(ann, out_path, CLASSES, CLASS_NUM=18):
    names = list(ann.keys())
    res = { CLASSES[cls]: [] for cls in range(CLASS_NUM) }
    for name in tqdm(names):
        result = ann[name]
        bboxes = result["bbox"]
        segs = result['vis']
        for cls in range(CLASS_NUM):
            curr_class = CLASSES[cls]
            bbox_cls = bboxes[cls]
            seg_cls = segs[cls]
            if( len(bbox_cls) > 0):
                for bbox, rle in zip(bbox_cls, seg_cls):
                    rle = np.array(rle)
                    xmin, ymin, xmax, ymax, score = bbox
                    location = list(rle.flatten())
                    location = [str(int(x)) for x in location]
                    if(score<0.05):
                        continue
                    out = name + " "+str(score)+ " "+" ".join(location)
                    res[curr_class].append(out)
    for key in res.keys():
        print(key)
        fp = open("./result/{}/".format(out_path)+key+".txt", 'w')
        print(len(res[key]))
        for line in res[key]:
            fp.write(line+"\n")
        fp.close()

def voc_ap(rec, prec, use_07_metric=False):
    """ ap = voc_ap(rec, prec, [use_07_metric])
      Compute VOC AP given precision and recall.
      If use_07_metric is true, uses the
      VOC 07 11 point method (default:False).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
        ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        # first appicend sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap
    
def eval_warpper(classname, detpath, ovthresh, coco):
    CLASS=['tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad']
    #CLASS={'tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad'}
    class_to_ind = dict(zip(CLASS, range(len(CLASS))))
    imgIds = coco.getImgIds()
    recs = {}
    use_07_metric = False
    for imgid in imgIds:
        img = coco.loadImgs(imgid)[0]
        file_name = img['file_name']
        file_name = file_name.split(".")[0]
        annIds = coco.getAnnIds(imgIds=[imgid], iscrowd=None)
        anns = coco.loadAnns(annIds)
        objects = []
        for ann in anns:
            obj = {}
            obj['name'] = CLASS[ann['category_id']]
            obj['bbox'] = ann['segmentation']
            objects.append(obj)
        recs[file_name] = objects
    class_recs = {}
    npos = 0
    for filename in list(recs.keys()):
        R = [obj for obj in recs[filename] if obj['name'] == classname]
        bbox = np.array([x['bbox'] for x in R])
        bbox = np.reshape(np.squeeze(bbox),(-1,4, 2))
        det = [False] * len(R)
        npos = npos + len(bbox)
        class_recs[filename] = {'bbox': bbox,
                                 'det': det}
    detfile = detpath.format(classname)
    with open(detfile, 'r') as f:
        lines = f.readlines()

    splitlines = [x.strip().split(' ') for x in lines]
    image_ids = [x[0] for x in splitlines]
    confidence = np.array([float(x[1]) for x in splitlines])
    BB = np.array([[float(z) for z in x[2:]] for x in splitlines])
    BB = np.reshape(BB, (-1, 4, 2))
    nd = len(image_ids)
    tp = np.zeros(nd)
    fp = np.zeros(nd)
    if BB.shape[0] > 0:
        # sort by confidence
        sorted_ind = np.argsort(-confidence)
        sorted_scores = np.sort(-confidence)
        BB = BB[sorted_ind, :]
        image_ids = [image_ids[x] for x in sorted_ind]

        # go down dets and mark TPs and FPs
        for d in tqdm(range(nd)):
            R = class_recs[image_ids[d]]
            bb = BB[d, :].astype(float).flatten()
            ovmax = -np.inf
            BBGT = R['bbox'].astype(float).reshape(-1,8)
            if BBGT.size > 0:
                # compute overlaps
                # intersection
                BBGT_xmin =  np.min(BBGT[:, 0::2], axis=1)
                BBGT_ymin = np.min(BBGT[:, 1::2], axis=1)
                BBGT_xmax = np.max(BBGT[:, 0::2], axis=1)
                BBGT_ymax = np.max(BBGT[:, 1::2], axis=1)
                bb_xmin = np.min(bb[0::2])
                bb_ymin = np.min(bb[1::2])
                bb_xmax = np.max(bb[0::2])
                bb_ymax = np.max(bb[1::2])

                ixmin = np.maximum(BBGT_xmin, bb_xmin)
                iymin = np.maximum(BBGT_ymin, bb_ymin)
                ixmax = np.minimum(BBGT_xmax, bb_xmax)
                iymax = np.minimum(BBGT_ymax, bb_ymax)
                iw = np.maximum(ixmax - ixmin + 1., 0.)
                ih = np.maximum(iymax - iymin + 1., 0.)
                inters = iw * ih

                # union
                uni = ((bb_xmax - bb_xmin + 1.) * (bb_ymax - bb_ymin + 1.) +
                       (BBGT_xmax - BBGT_xmin + 1.) *
                       (BBGT_ymax - BBGT_ymin + 1.) - inters)

                overlaps = inters / uni

                BBGT_keep_mask = overlaps > 0
                BBGT_keep = BBGT[BBGT_keep_mask, :]
                BBGT_keep_index = np.where(overlaps > 0)[0]
                
                
                
                def calcoverlaps(BBGT_keep, bb):
                    overlaps = []
                    for index, GT in enumerate(BBGT_keep):

                        overlap = polyiou.iou_poly(polyiou.VectorDouble(BBGT_keep[index]), polyiou.VectorDouble(bb))
                        overlaps.append(overlap)
                    return overlaps
                if len(BBGT_keep) > 0:
                    overlaps = calcoverlaps(BBGT_keep, bb)

                    ovmax = np.max(overlaps)
                    jmax = np.argmax(overlaps)
                    # pdb.set_trace()
                    jmax = BBGT_keep_index[jmax]

            if ovmax > ovthresh:
                if not R['det'][jmax]:
                    tp[d] = 1.
                    R['det'][jmax] = 1
                else:
                    fp[d] = 1.
            else:
                fp[d] = 1.
    fp = np.cumsum(fp)
    tp = np.cumsum(tp)
    rec = tp / float(npos)
    # avoid divide by zero in case the first detection matches a difficult
    # ground truth
    prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
    ap = voc_ap(rec, prec, use_07_metric)
    print("ap of {} is {}".format(classname, ap))
    return ap

def evaluate(iou_thresh):
    p = Pool(18)
    detpath="./result/val_temp/{}.txt"
    CLASS=['tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad']
    iou_thresh = 0.5
    aps = []
    coco=COCO("/home/xfr/rssid/data/annotation/annos_rscup_val.json")
    for classname in tqdm(CLASS):
        aps.append(p.apply_async(eval_warpper, args=(classname, detpath, iou_thresh, coco)))
    ret = []
    for ap in aps:
        ret.append(ap.get())
    p.close()
    p.join()
    print("map is {}".format(np.mean(np.array(ret))))
    


In [84]:
CLASSES = ['tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane', 'ground-track-field', 'helicopter', 'airport', 'harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field', 'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad']

### val

In [87]:
config_file = "configs/rs_cascade_mask_rcnn_r50_fpn_ohem.py"
result_file = "./result/val.pkl"
anno_file = "/home/xfr/mmdetection/data/rscup/annotation/annos_rscup_val.json"
out_file = "./result/eval_temp.pkl"
img_prefix = "./data/rscup/val/"
ann = merge_result(config_file, result_file, anno_file, img_prefix,out_file)
ann = nms(ann, "poly", 0.5)
generate_submit(ann, "val_temp", CLASSES)
evaluate(0.5)

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


tennis-court
1181
container-crane
78
storage-tank
4153
baseball-diamond
579
plane
4120
ground-track-field
564
helicopter
263
airport
200
harbor
5862
ship
20034
large-vehicle
15914
swimming-pool
1934
soccer-ball-field
439
roundabout
808
basketball-court
505
bridge
2221
small-vehicle
59188
helipad
2
loading annotations into memory...
Done (t=1.02s)
creating index...
index created!


ap of container-crane is 0.0047619047619047615
ap of baseball-diamond is 0.7446413422470144
ap of tennis-court is 0.9471169315909007
ap of helicopter is 0.5748580643083115
ap of ground-track-field is 0.7363699323182413
ap of plane is 0.9014730448234975
ap of storage-tank is 0.6342319648222312
ap of airport is 0.8051250130017995
ap of soccer-ball-field is 0.5671092724060538
ap of swimming-pool is 0.5833886328391472
ap of basketball-court is 0.6469895378422549
ap of roundabout is 0.6353624515483904
ap of harbor is 0.7345451172304186
ap of helipad is 0.0
ap of bridge is 0.46401781732210945
ap of large-vehicle is 0.7695623382535197
ap of ship is 0.86022912425063
ap of small-vehicle is 0.4679054598372296
map is 0.6154271083002031


In [88]:
ann = mmcv.load(out_file)
ann = nms(ann, "poly", 0.3)
generate_submit(ann, "val_temp", CLASSES)
evaluate(0.5)

tennis-court
1102
container-crane
61
storage-tank
3844
baseball-diamond
549
plane
3812
ground-track-field
547
helicopter
252
airport
175
harbor
4656
ship
18284
large-vehicle
14589
swimming-pool
1714
soccer-ball-field
413
roundabout
780
basketball-court
418
bridge
1953
small-vehicle
53465
helipad
2
loading annotations into memory...
Done (t=0.89s)
creating index...
index created!


ap of container-crane is 0.00510204081632653
ap of tennis-court is 0.9477306277413307
ap of ground-track-field is 0.7367738171638177
ap of storage-tank is 0.6309718375248123
ap of helicopter is 0.5750325434055158
ap of baseball-diamond is 0.7398826364403034
ap of airport is 0.7958449127063089
ap of plane is 0.9013933869606383
ap of harbor is 0.7274291536247965
ap of swimming-pool is 0.5800966177744742
ap of soccer-ball-field is 0.5662417701496385
ap of large-vehicle is 0.7691954772537981
ap of basketball-court is 0.6560094183878675
ap of helipad is 0.0
ap of bridge is 0.46040882467261635
ap of roundabout is 0.6332201109585067
ap of ship is 0.858962794214532
ap of small-vehicle is 0.4674851452702723
map is 0.6139878397258642


### Test

In [89]:
config_file = "configs/rs_cascade_mask_rcnn_r50_fpn_ohem.py"
result_file = "./result/test.pkl"
anno_file = "/home/xfr/mmdetection/data/rscup/annotation/annos_rscup_test.json"
out_file = "./result/test_temp.pkl"
img_prefix = "./data/rscup/test/"
ann = merge_result(config_file, result_file, anno_file, img_prefix, out_file)


loading annotations into memory...
Done (t=0.23s)
creating index...
index created!
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [96]:
ann = mmcv.load(out_file)
ann = nms(ann, "poly", 0.5)
generate_submit(ann, "detection", CLASSES)

tennis-court
1133
container-crane
1364
storage-tank
10942
baseball-diamond
378
plane
1687
ground-track-field
1189
helicopter
21
airport
892
harbor
5724
ship
26390
large-vehicle
8353
swimming-pool
1546
soccer-ball-field
794
roundabout
1747
basketball-court
664
bridge
4360
small-vehicle
44224
helipad
62


### visualization

In [93]:
# result2 = mmcv.load("/home/xiongbaiqiao/competition/detection/test_postmerge.pkl")
def vis_result(name):
    datadir = "/home/xfr/rssid/data/test/images/"
    files = os.listdir(datadir)
    img_name = ""
    for file in files:
        img_name = file.split(".")[0]
        if(img_name == name):
            break
    filename = os.path.join(datadir, file)
    img = cv2.imread(filename)
    result = result2[name]
    bboxes = result["bbox"]
    segs = result['vis']
    ret = { CLASSES[cls]: [] for cls in range(CLASS_NUM) }
    for cls in range(CLASS_NUM):
        curr_class = CLASSES[cls]
        bbox_cls = bboxes[cls]
        seg_cls = segs[cls]
        if( len(bbox_cls) > 0):
            for bbox, rle in zip(bbox_cls, seg_cls):
                rle = np.array(rle)
                xmin, ymin, xmax, ymax, score = bbox
                location = list(rle.flatten())
                location = [str(int(x)) for x in location]
                if(score<0.05):
                    continue
                out = name + " "+str(score)+ " "+" ".join(location)
                ret[curr_class].append(out)
                cv2.putText(img, CLASSES[cls] + str(score), (int(xmin), int(ymin)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                cv2.polylines(img, np.array([rle], np.int32), 1, (0, 255, 0), 1)
                #cv2.poly(img, np.array([rle], np.int32), (0,))
    cv2.imwrite("./result/test2/{}.jpg".format(name), img)
    return ret
    
result2 = ann
CLASS_NUM=18
names = list(result2.keys())
res = { CLASSES[cls]: [] for cls in range(CLASS_NUM) }
p = Pool(18)
m = []
pbar = tqdm(total=len(names))
def update(*a):
    pbar.update()
for name in names:
    m.append(p.apply_async(vis_result, args=([name]), callback=update))
p.close()
p.join()
for ret in m:
    temp = ret.get()
    for cls in CLASSES:
        res[cls] += temp[cls]
for key in res.keys():
    print(key)
    print(len(res[key]))

tennis-court
1133
container-crane
1364
storage-tank
10942
baseball-diamond
378
plane
1687
ground-track-field
1189
helicopter
21
airport
892
harbor
5724
ship
26390
large-vehicle
8353
swimming-pool
1546
soccer-ball-field
794
roundabout
1747
basketball-court
664
bridge
4360
small-vehicle
44224
helipad
62
